In [4]:
import numpy as np
import librosa
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import mysql.connector
import io
from fastdtw import fastdtw

In [5]:
conn = mysql.connector.connect(
            host='localhost',  # Replace with your MySQL host
            user='root',  # Replace with your MySQL username
            password='Sam@9818',  # Replace with your MySQL password
            database='mail_db' ,
            port= '3306'# Replace with your MySQL database name
        )

In [6]:
cursor = conn.cursor()


In [7]:
# Step 1: Define a function to extract MFCC features from an audio file
def extract_mfcc_features(audio_file):
    # Load the audio file
    audio, sr = librosa.load(audio_file)
    
    # Extract MFCC features
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    
    return mfccs


In [8]:
def fetch_audio_data(email_id):
    query = "SELECT audio_1, audio_2, audio_3 FROM user_data WHERE email_id = %s"
    cursor.execute(query, (email_id,))
    row = cursor.fetchone()
    return row

In [9]:
email_id="sam@gmail.com"
audio_data = fetch_audio_data(email_id)

In [10]:
# Step 2: Replace the file paths with the actual paths to your audio recordings
enrollment1_features = extract_mfcc_features(io.BytesIO(audio_data[0]))
enrollment2_features = extract_mfcc_features(io.BytesIO(audio_data[1]))
enrollment3_features = extract_mfcc_features(io.BytesIO(audio_data[2]))

In [22]:
# Extract MFCC features for the test audio sample
test_features = extract_mfcc_features("D:\\dataset\\tushar2.wav")

In [23]:
# Step 3: Prepare the Data
# Organize your data into training and test sets

# Find the maximum number of time steps among all speakers
max_time_steps = max(enrollment1_features.shape[1], enrollment2_features.shape[1], enrollment3_features.shape[1])

# Pad or truncate the enrollment arrays to the same shape
enrollment1_features = np.pad(enrollment1_features, ((0, 0), (0, max_time_steps - enrollment1_features.shape[1])), mode='constant')
enrollment2_features = np.pad(enrollment2_features, ((0, 0), (0, max_time_steps - enrollment2_features.shape[1])), mode='constant')
enrollment3_features = np.pad(enrollment3_features, ((0, 0), (0, max_time_steps - enrollment3_features.shape[1])), mode='constant')

# Now, you can create X_train from these padded/truncated arrays
X_train = np.array([enrollment1_features, enrollment2_features, enrollment3_features])
y_train = np.array([0, 1, 2])  # Assign unique labels to each speaker (0, 1, 2 in this example)


In [24]:
# Step 4: Define the CNN Model
model = keras.Sequential([
    layers.Input(shape=(X_train.shape[1], X_train.shape[2])),
    layers.Conv1D(64, kernel_size=3, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(np.unique(y_train)), activation='softmax')
])

In [25]:
# Step 5: Compile the Model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [26]:
# Step 6: Train the Model
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
1/1 [==============================] - 1s 541ms/step - loss: 47.1680 - accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 9ms/step - loss: 27.1020 - accuracy: 0.6667
Epoch 3/10
1/1 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 9ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 8ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 9ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 7ms/step - loss: 8.3446e-07 - accuracy: 1.0000
Epoch 8/10
1/1 [==============================] - 0s 7ms/step - loss: 3.1188 - accuracy: 0.6667
Epoch 9/10
1/1 [==============================] - 0s 8ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - accuracy: 1.0000


In [27]:
# Step 7: Verification
# Predict the label for the test audio sample
predicted_label = np.argmax(model.predict(X_train))

1/1 [==============================] - 0s 58ms/step


In [28]:
# # Step 8: Save the model in the .h5 format
# model.save('model.h5')

In [30]:
# Map the predicted label to the corresponding speaker
speakers = {0: "Speaker 1", 1: "Speaker 2", 2: "Speaker 3"}
if predicted_label in speakers:
    print(f"{speakers[predicted_label]} verified.")
else:
    print("Speaker not recognized.")

Speaker 1 verified.


pygame 2.5.2 (SDL 2.28.3, Python 3.11.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
